# Classification Model
* File Name: Reference_implementation of BFSI 
* Date of creation(dd-mm-yyyy) : 27-03-2023
* Author Name/Dept : AIShield
* Organization : BGSW
* Description : Source Code of reference implementation of AIShield API
* Copyright : Copyright 2023 Bosch Global Software Technologies Private Limited. All Rights Reserved.

### Metadata
* Dataset: Banking Marketing Campagin
* Size of dataset:41188, 18
* Number of class : 2
* Original Model: XGBOOST 

### Outcomes
* Accuracy of model: 0.92


<a target="_blank" href="https://colab.research.google.com/github/bosch-aisecurity-aishield/Reference-Implementations/blob/main/Product_Taskpair_wise/Tabular_Classification/evasion/Evasion_Reference_Implementation_BFSI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
"""
Description: commands to install all the packages, remove comments to install all the libraries
"""
# ! pip install xgboost==1.6.2
# ! pip install pandas==1.1.5
# ! pip install scikit-learn==1.0.2
# ! pip install numpy==1.22
# ! pip install pyminizip
# !pip install requests==2.28.0
# !pip install humanfriendly==9.2

'\nDescription: commands to install all the packages, remove comments to install all the libraries\n'

### 1.0 Import libraries

In [3]:
import os
import copy
import json
import time
import pickle
import shutil
import requests
import py_compile
import pyminizip
import zipfile
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


### 2.0 Data Loading and Preprocessing

#### Download dataset from the link https://archive.ics.uci.edu/dataset/222/bank+marketing  
1. then download the bank-additional.zip file and extract it , If running in colab, then uncomment the below cell to upload the downloaded zip file to colab

In [2]:
# # To upload the downloaded zip file in Colab, please uncomment the below cell.
# from google.colab import files
# import zipfile
# import io

# uploaded = files.upload()

# def extract_nested_zip(zip_file):
#     # Look for nested zip files
#     nested_zip_names = [name for name in zip_file.namelist() if name.endswith(".zip")]

#     if not nested_zip_names:
#         print("No nested zip files found.")
#         return

#     for nested_zip_name in nested_zip_names:
#         with zip_file.open(nested_zip_name) as nested_zip_file:
#             with io.BytesIO(nested_zip_file.read()) as nested_buffer:
#                 with zipfile.ZipFile(nested_buffer, 'r') as nested_zip:
#                     # Extract the contents of the nested zip file
#                     nested_zip.extractall()

# for name, content in uploaded.items():
#     with io.BytesIO(content) as buffer:
#         with zipfile.ZipFile(buffer, 'r') as outer_zip:
#             # Extract the contents of the outer zip file
#             outer_zip.extractall()

#             # Extract nested zip files recursively
#             extract_nested_zip(outer_zip)

# print("Extraction completed.")


In [3]:
'''
Description : Loading credit card fraud detection dataset
'''
df = pd.read_csv('bank-additional/bank-additional-full.csv',delimiter=";")

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [5]:
df.replace(['basic.6y','basic.4y', 'basic.9y'], 'basic', inplace=True)

In [6]:
columns = ['day_of_week','month']
df.drop(columns, axis=1, inplace=True)

In [7]:
'''Description : Label Encoding the Categorical Columns'''

le =LabelEncoder()
df.job = le.fit_transform(df.job)
df.marital = le.fit_transform(df.marital)
df.education = le.fit_transform(df.education)
df.housing = le.fit_transform(df.housing)
df.loan = le.fit_transform(df.loan)
df.poutcome = le.fit_transform(df.poutcome)
df.contact = le.fit_transform(df.contact)
df.default = le.fit_transform(df.default)
df.y = le.fit_transform(df.y)

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,1,1,0,0,2,0,0,487,2,999,0,1,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,7,2,1,0,0,0,1,346,4,999,0,1,1.1,93.994,-36.4,4.855,5191.0,0
2,25,7,1,1,0,2,0,1,227,1,999,0,1,1.4,94.465,-41.8,4.962,5228.1,0
3,38,7,1,0,0,1,1,1,17,3,999,0,1,1.4,94.465,-41.8,4.959,5228.1,0
4,47,0,1,4,0,2,0,0,58,1,999,0,1,-0.1,93.200,-42.0,4.191,5195.8,0


In [9]:
output = 'y'
 
X = df.loc[:, df.columns != output]
y = df['y']

In [10]:
'''
Description : Splitting data for validation
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [11]:
'''
Description : Check size of dataset
'''
print("shape of x_train: ", X_train.shape)
print("shape of y_train: ", y_train.shape)
print("shape of x_test: ", X_test.shape)
print("shape of y_test: ", y_test.shape)

shape of x_train:  (3089, 18)
shape of y_train:  (3089,)
shape of x_test:  (1030, 18)
shape of y_test:  (1030,)


In [12]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

### 3.0 Model Development and Training

In [13]:
def make_directory(directory):
    """
    create directory

    Parameters
    ----------
    directorys : list containing the directorys path to create 
    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

In [14]:
def delete_directory(directorys):
    """
    delete directory 

    Parameters
    ----------
    directorys : list containing the directorys to deleate along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
                

In [15]:
def make_archive(base_name,root_dir,zip_format='zip'):
    """
    created zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar 
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)
    

In [3]:
"Description : Create data, model and label folder"
data_path=os.path.join(os.getcwd(),"data")
model_path=os.path.join(os.getcwd(),"model")
minmax_path=os.path.join(os.getcwd(),"minmax")
zip_path=os.path.join(os.getcwd(),"zip")
pyc_model_path=os.path.join(os.getcwd(),"pycmodel")
report_path = os.path.join(os.getcwd(), "reports")
#deleting folder
delete_directory(directorys=[data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

#creating folder
make_directory([data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

'Description : Create data, model and label folder'

In [17]:
def model():
    XG = XGBClassifier()
    return XG

In [18]:
"""
Description : Create architecture of the model
"""
model=model()
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


In [19]:
"""
Description: Trainig the model 
"""

model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [20]:
"""
Saving the model under model directory
"""
modelname = "xgboost_evasion.pkl"
pickle.dump(model, open(model_path+"/"+modelname, 'wb'))

In [21]:
"""
Description: Get prediction on validation data
"""
pred=model.predict(x_test)

In [22]:
print('Accuracy score of the model is {}'.format(accuracy_score(y_test, pred)))
print('F1 score of the model is {}'.format(f1_score(y_test, pred)))
print('Confusion Matrix \n {}'.format(confusion_matrix(y_test, pred)))
print('Classification Report \n {}'.format(classification_report(y_test, pred)))

Accuracy score of the model is 0.9155339805825242
F1 score of the model is 0.5347593582887701
Confusion Matrix 
 [[893  37]
 [ 50  50]]
Classification Report 
               precision    recall  f1-score   support

           0       0.95      0.96      0.95       930
           1       0.57      0.50      0.53       100

    accuracy                           0.92      1030
   macro avg       0.76      0.73      0.74      1030
weighted avg       0.91      0.92      0.91      1030



### 4.0 Prepare Data , Model and MinMax

In [23]:
"""
Description: Zip data
"""
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
number_of_sample = 100

final_df = pd.DataFrame(list(x),columns=df.columns[:-1])
final_df[output] = y
final_df[:number_of_sample].to_csv(os.path.join(data_path,"banking_evasion.csv"),index=False)
make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')

In [24]:
"""Prepare MinMax"""

min_values = final_df.min().to_numpy()
max_values = final_df.max().to_numpy()
x= np.array([min_values,max_values])
df_m = pd.DataFrame(x,columns=final_df.columns)
df_m.to_csv(minmax_path+"/minmax.csv",index=False)

In [25]:
"""
Description: Zip Minmax
"""
make_archive(base_name=os.path.join(zip_path,"minmax"),root_dir=minmax_path,zip_format='zip')

In [26]:
"""
Description: Create encrypted Model
"""
def create_model_pyc(model, model_framework,pyc_model_path):
    
    if model_framework.lower() == "scikit-learn":

        python_code ='''
#importing libraries
import pickle
import zipfile
import os

"""
    class for base model
"""
#define class
class BaseModel():
    def __init__(self,model_path=""):

        """
        constructor for class

        Parameters
        ----------
        input_shape : TYPE, optional
            DESCRIPTION. The default is (100,1).
        Returns
        -------
        None.

        """
        self.model_path = model_path
        
    def load_protected_pickleModel(self,filename,password,picklemodelname):
      

        """
        model architecture

        Parameters
        ----------
        filename : string
            DESCRIPTION.zipped protected filename
            
        password : String
            DESCRIPTION.zipped file password
        
        picklemodelname  :String
            DESCRIPTION.pickle file name present in a ziiped protected
            
        Returns
        -------
        model : model
            DESCRIPTION.

        """
        filepath = os.path.join(self.model_path,filename)
        #print(filepath)
        #print(filename)
        with zipfile.ZipFile(filepath, 'r') as file:
            with file.open(picklemodelname,'r',pwd = bytes(password, 'utf-8')) as f:
                pck = pickle.load(f)
        return pck
    
    def predict(self,X):

        """
        predict for given data

        Parameters
        ----------
        X : numpy array 
            DESCRIPTION.

        Returns
        -------
        pred : numpy array
            DESCRIPTION.

        """
        filename = "modelencrypt.zip"
        password = "987654321"
        picklemodelname = "xgboost_evasion.pkl"
        model = self.load_protected_pickleModel(filename=filename,password=password,picklemodelname=picklemodelname)
        pred = model.predict(X)
        return pred'''
        # Writing to file
        with open("base_model.py", "w") as file:
            # Writing data to a file
            file.writelines(python_code)
         
        """
        Description: function to create .pyc file
        """
        py_compile.compile(file="base_model.py",cfile=pyc_model_path+'/base_model.pyc')
        
        """
        Description: function to create zipped password protected pickle model file
        """

        def zip_model(input_path,output_path,password,com_lvl=5):
            pyminizip.compress(input_path, None, output_path,
                               password, com_lvl)
            
        zip_model(input_path =os.path.join(model_path,modelname),output_path = pyc_model_path+"/modelencrypt.zip" ,password = "987654321",com_lvl=5)
            

In [27]:
"""
Description: Creating encrypted Model
"""

input_dimension = (number_of_sample,final_df.shape[1]-1)
number_of_classes = 2
model_framework = "scikit-learn"
model_encryption = 0 #0 for non encrypted file and 1 for encrypted (pyc) 
create_model_pyc(model,model_framework, pyc_model_path)

In [28]:
"""
Description: Zip model
"""
 # model_encryption 0 if model is uploaded directly as a zip, model_encryption 1 if model is encryted as .pyc and uploaded as a zip
if os.path.isfile(os.path.join(zip_path,"model.zip")):
    delete_directory(directorys=[os.path.join(zip_path,"model.zip")])
if model_encryption:
    make_archive(base_name=os.path.join(zip_path,"bfsi_pyc_model"),root_dir=pyc_model_path,zip_format='zip')
else:
    make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

### 5.0 AIShield API Call


In [29]:
"""
Description: AIShield API call
"""

url="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
headers={'Cache-Control': 'no-cache'}
headers["Org-Id"] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
headers['x-api-key'] =  "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

#### 5.1 Model Registration


In [4]:
"""
Description: call Model registration api to get unique model it and url to upload data, model and label
"""
model_registration_url = url + "/model_registration/upload"
model_registration_payload = {
    'task_type':"TC",
    "analysis_type": "EVA"
}
new_request = requests.request(method="POST", url=model_registration_url, headers=headers, json=model_registration_payload)
new_request=json.loads(new_request.text)
model_id = new_request["data"]['model_id']
data_upload_url = new_request["data"]["urls"]['data_upload_url']
label_upload_url = new_request["data"]["urls"]['minmax_upload_url']
model_upload_url = new_request["data"]["urls"]['model_upload_url']

print('model_id: ', model_id)

model_id:  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [31]:
"""
Description: Files path
"""
data_path=os.path.join(zip_path,'data.zip')  # full path of data zip
label_path=os.path.join(zip_path,'minmax.zip') # full path of label zip
# model_path=os.path.join(zip_path,'bfsi_pyc_model.zip') # uncomment if model_encryption = 1 (pyc)
model_path=os.path.join(zip_path,'model.zip') # uncomment if model_encryption = 0 (unencrypted)

In [32]:
def upload_file(url, file_path):
    """
    url: URL to upload
    file_path: file to be uploaded
    """
    new_request = requests.request(method="PUT", url=url, data=open(file_path,'rb'))
    status_cd = new_request.status_code
    if status_cd == 200:
        status = 'upload sucessful'
    else:
        status = 'upload failed'
    return status

In [33]:
"""
Description: Hit AIShield File Upload API
"""
data_upload_status = upload_file(data_upload_url, data_path)
label_upload_status = upload_file(label_upload_url, label_path)
model_upload_status = upload_file(model_upload_url, model_path)

print('data_upload_status: ', data_upload_status)
print('label_upload_status: ', label_upload_status)
print('model_upload_status: ', model_upload_status)

data_upload_status:  upload sucessful
label_upload_status:  upload sucessful
model_upload_status:  upload sucessful


#### 5.2 Model Analysis


In [34]:
payload = {
            "categorical_columns_info": "job,marital,education,default,housing,loan,contact,poutcome",
            "defense_bestonly": "no",
            "encryption_strategy": model_encryption,
            "input_dimensions": str(input_dimension),
            "is_category_columns": "yes",
            "model_api_details": "na",
            "model_framework": "scikit-learn",
            "normalize_data": "yes",
            "number_of_classes": number_of_classes,
            "use_model_api": "no"
        }

In [5]:
"""
Description: Hit AIShield VulnerabilityReport api
"""
model_analysis_url = url + "/model_analyse/"+model_id
if data_upload_status == "upload sucessful" and model_upload_status == "upload sucessful" and label_upload_status == "upload sucessful":
    new_request = requests.request(method="POST", url=model_analysis_url, json=payload,headers=headers)
    new_request=json.loads(new_request.text)
    for k, v in new_request.items():
        print("* {} : {}".format(k,v))

* api_version : 1.5
* job_id : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
* monitor_link : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [6]:
"""
Description: Get job id from api response
"""
job_id=new_request['job_id']
print(f"Job id : {job_id}")

Job id : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [37]:
def monitor_api_progress(new_job_id):
        job_status_url = url + "/job_status_detailed?job_id=" + new_job_id

        # status dictionary
        status_dictionary = {
            'ModelExploration_Status': 'na',
            'SanityCheck_Status': 'na',
            'QueryGenerator_Status': 'na',
            'VunerabilityEngine_Status': 'na',
            'EvasionAnalysis_Status':'na',
            'DefenseReport_Status': 'na',
        }
        counts = [0] * len(status_dictionary)
        failed_api_hit_count = 0
        while True:
            time.sleep(2)
            try:
                job_status_response = requests.request("GET", job_status_url, params={},
                                                       headers=headers)

                job_status_payload = json.loads(job_status_response.text)
                failing_key = 'ModelExploration_Status'
                for i, key in enumerate(status_dictionary.keys()):
                    if status_dictionary[key] == 'na':
                        if job_status_payload[key] == 'inprogress' and status_dictionary[key] == 'na':
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                        elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                            status_dictionary[key] = job_status_payload[key]
                            counts[i] += 1
                            print(str(key), ":", status_dictionary[key])

                        if job_status_payload[key] == 'failed':
                            failing_key = key
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                    elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                        status_dictionary[key] = job_status_payload[key]
                        if counts[i] < 1:
                            print(str(key), ":", status_dictionary[key])
                        counts[i] += 1

                    else:
                        if job_status_payload[key] == 'failed':
                            failing_key = key
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                if job_status_payload[failing_key] == 'failed':
                    break

                if status_dictionary['VunerabilityEngine_Status'] == 'passed' or status_dictionary[
                    'VunerabilityEngine_Status'] == 'completed' and job_status_payload[
                    'CurrentStatus'] == "Defense generation is not triggered":
                    print("\n Vulnerability score {} failed to cross vulnerability threshold of {}".format(
                        job_status_payload['VulnerabiltyScore'], payload['vulnerability_threshold']))
                    break
                if job_status_payload['DefenseReport_Status'] == 'completed':
                    break
            except Exception as e:
                failed_api_hit_count += 1
                print("Error {}. trying {} ...".format(str(e), failed_api_hit_count))
                if failed_api_hit_count >= 3:
                    break
        return status_dictionary

In [38]:
"""
Description: Continuos monitoring of jod progress
"""
status_dictionary = monitor_api_progress(new_job_id=job_id)

ModelExploration_Status : completed
SanityCheck_Status : passed
QueryGenerator_Status : completed
VunerabilityEngine_Status : completed
EvasionAnalysis_Status : inprogress
EvasionAnalysis_Status : completed
DefenseReport_Status : completed


In [39]:
def download_artifact(job_id, report_type='Vulnerability', file_format=0):
    """
    job_id: job_id  received after successful api call
    report_type: report to be downloaded
    file_format: change file_format to : 0- all report in zip 
                        1- report in .txt 
                        2- report in .pdf
                        3- report in .json
                        4- report in .xml
    """
    report_url = url + "/get_report?job_id=" + str(
        job_id) + "&report_type=" + report_type + "&file_format=" + str(file_format)

    headers1=headers
    headers1["content-type"]= "application/zip"

    response = requests.request("GET", report_url, params={}, headers=headers1)

    if file_format == 0 or "Attack_samples":
        with open(os.path.join(report_path, report_type + ".zip"), 'wb') as f:
            f.write(response.content)
    elif file_format == 1:
        with open(os.path.join(report_path, report_type + ".txt"), 'wb') as f:
            f.write(response.content)
    elif file_format == 2:
        with open(os.path.join(report_path, report_type + ".pdf"), 'wb') as f:
            f.write(response.content)
    elif file_format == 3:
        with open(os.path.join(report_path, report_type + ".json"), 'wb') as f:
            f.write(response.content)
    elif file_format == 4:
        with open(os.path.join(report_path, report_type + ".xml"), 'wb') as f:
            f.write(response.content)

In [40]:
"""
Description: download generated artifact
"""
if status_dictionary["VunerabilityEngine_Status"] == 'completed':
    download_artifact(job_id=job_id, report_type='Vulnerability', file_format=0) 
    download_artifact(job_id=job_id, report_type='Attack_samples', file_format=0)

if status_dictionary["DefenseReport_Status"] == 'completed':
    download_artifact(job_id=job_id, report_type='Defense', file_format=0)
    download_artifact(job_id=job_id, report_type='Defense_artifact', file_format=0)

In [41]:
def zip_extractor(file, extract_path=None, delete_zip=False):
    """
    extract zip file to the given path

    Parameters
    ----------
    file : path of zip file
    extract_path : path to extract zip file, default considered parent directory
    delete_zip: True, delete zip file after unzipping it

    Returns
    -------
    None.
    """
    if extract_path is None:
        extract_path = os.path.dirname(file)
    print("Extracting : {}".format(file))
    zf = zipfile.ZipFile(file=file, mode='r')
    zf.extractall(extract_path)
    zf.close()
    if delete_zip:
        os.remove(file)
        print("{} removed successfully.".format(file))


In [7]:
"""
Description: Extracting defense artifact
"""
zip_extractor(file=os.path.join(report_path, 'Defense_artifact.zip'))

'\nDescription: Extracting defense artifact\n'

In [8]:
"""
Description: Extracting attack sample
"""
zip_extractor(file=os.path.join(report_path, 'Attack_samples.zip'))

'\nDescription: Extracting attack sample\n'

In [9]:
"""
Description: Load defense model
"""
defense_model_path = os.path.join(report_path, 'defense_model.pkl')
defense_model = pickle.load(open(defense_model_path,'rb'))

'\nDescription: Load defense model\n'

In [45]:
"""
Description: Use defense model
"""
from reports import predict
defense = predict.AISDefenseModel(defense_model,model_framework="scikit-learn")

In [47]:
"""
Description: Pass Original sample data to get prediction
"""
label, prob = defense.predict(x_test[:5])
print("label: {}".format(label))
print("prob: {}".format(prob))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

label: ['not-attack' 'not-attack' 'not-attack' 'not-attack' 'not-attack']
prob: [0. 0. 0. 0. 0.]


In [4]:
"""
Description: attack_data visualization
"""

attack_data = pd.read_csv(os.path.join(report_path,"attack_samples.csv"))

In [5]:
attack_data.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0.093354,0.730612,0.653131,0.791850,0.0,0.990764,0.996411,0.992500,0.121563,0.000000,0.991511,0.000000,0.502935,0.480186,0.776534,0.550559,0.035653,0.000000
1,0.121005,0.000000,0.693062,0.801084,0.0,1.000000,0.000000,0.000000,0.317266,0.002381,1.000000,0.000000,0.511468,0.663929,0.358310,0.340315,0.776495,0.843138
2,0.525601,0.009095,0.309480,0.199172,0.0,0.982245,0.003929,0.982974,0.274631,0.000000,0.973405,0.019302,0.514480,0.912410,0.618720,0.524683,0.990246,0.809770
3,0.031091,0.016219,0.627101,0.563320,0.0,0.003398,0.029189,0.009059,0.259990,0.000000,0.972987,0.000000,0.502052,0.956374,0.623677,0.320410,0.963832,0.956310
4,0.458831,0.178124,0.000000,0.776932,0.0,0.017754,0.025488,0.014317,0.256228,0.000000,0.990683,0.000000,0.498007,0.680622,0.364213,0.343457,0.787258,0.827779
